In [1]:
import pyspark
import os

from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'

app_name = "lab2"

sc = SparkSession.builder.appName(app_name).master("local[*]").getOrCreate()

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [3]:
pro_lang = sc.read.csv("programming-languages.csv")
posts_sample = sc.read.format("xml").options(rowTag="row").load('posts_sample.xml')

Создаем список имен ЯП

In [4]:
pro_lang_list = [str(x[0]) for x in pro_lang.collect()]

pro_lang_list[:10]

['name',
 'A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET']

Функция language_detect() производит перевод текста в нижний регистр и поиск названия ЯП в каждой строке, если язык был найден, то создается кортеж, иначе None.

 Функция check_date() для проверки даты

In [5]:
def language_detect(x):
  lang_tag = None
  for language in pro_lang_list:
    if "<" + language.lower() + ">" in x._Tags.lower():
      lang_tag = language
      break
  if lang_tag is None:
    return None
  return (x._Id, lang_tag)

In [6]:
def check_date(x, year):
  start = datetime(year=year, month=1, day=1)
  end = datetime(year=year, month=12, day=31)
  CreationDate = x._CreationDate
  return CreationDate >= start and CreationDate <= end


1.   Убираем пустые значения и оставляет диапазон с 2010 по 2020 год
2.   Находим ЯП в каждой строке и убираем пустые значения, если не был найден
1.   Считаем сколько раз упоминался каждый язык программирования в каждом годе и сортируем по количеству повторений
2.   Сортировка по количеству упоминаний

In [9]:
result = {}

for year in range(2010, 2020):
  result[year] = posts_sample.rdd\
      .filter(lambda x: x._Tags is not None and check_date(x, year))\
      .map(language_detect)\
      .filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1])\
      .aggregateByKey(
          0,
          lambda x, y: x + 1,
          lambda x1, x2: x1 + x2,
      )\
      .sortBy(lambda x: x[1], ascending=False)\
      .toDF()

  result[year] = result[year].select(col("_1").alias("ЯП"), 
                                                 col("_2").alias(f"Кол-во_упоминаний_в_{year}")).limit(10)
  result[year].show()

+-----------+------------------------+
|         ЯП|Кол-во_упоминаний_в_2010|
+-----------+------------------------+
|       Java|                      52|
| JavaScript|                      44|
|        PHP|                      42|
|     Python|                      25|
|Objective-C|                      22|
|          C|                      20|
|       Ruby|                      11|
|     Delphi|                       7|
|          R|                       3|
|       Bash|                       3|
+-----------+------------------------+

+-----------+------------------------+
|         ЯП|Кол-во_упоминаний_в_2011|
+-----------+------------------------+
|        PHP|                      97|
|       Java|                      92|
| JavaScript|                      82|
|     Python|                      35|
|Objective-C|                      33|
|          C|                      24|
|       Ruby|                      17|
|     Delphi|                       8|
|       Perl|           

In [10]:
for i in result.keys():
    result[i].write.format("parquet").save(f"year_{i}")